In [1]:
from __future__ import print_function, division
import sys
sys.path.append('../')
import os
import os.path

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
#import matplotlib.pyplot as plt
import time as t
import pandas as pd
from datetime import datetime


# Matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import cv2 # for image manipulation

import torch.utils.data as data
import glob
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import json

In [2]:
# Mean and std are as obtained from Autoencoder notebook,
cifar10_mean = [ 0.4732,  0.4637,  0.4293]
cifar10_std = [ 0.2202,  0.2168,  0.2153]

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(cifar10_mean, cifar10_std)
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(cifar10_mean, cifar10_std)
    ]),
}

# Define dataloader

In [3]:
class Read_Dataset():
    
    def __init__(self, file_path,transform=None):
        self.data = pd.read_csv(file_path, sep = ',')
        self.label = self.data['class_label'].tolist() 
        self.image = [np.asarray(json.loads(x)) for x in self.data['img_data']]
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        label = self.label[index]
        image_tensor = torch.from_numpy(self.image[index].astype('float32'))
        image = transforms.ToPILImage()(image_tensor)
        
        if self.transform is not None:
            image = self.transform(image)
            
        return image, label

train_file_name = './modified_img_data.csv' 
test_file_name = './modified_img_data_test.csv'

train_val_list = ['train' , 'val']
path_to_input_file = {
    'train' : train_file_name ,
    'val':  test_file_name
    }

In [4]:
start = t.time()
image_datasets = {x: Read_Dataset( file_path = path_to_input_file[x], transform = data_transforms[x])
                  for x in train_val_list}
print("time taken for reading data sets : ", str(t.time() - start))

time taken for reading data sets :  80.51884317398071


In [5]:
batch_size_num = 64
dataloders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size_num, shuffle=True, num_workers=4)
              for x in train_val_list}

# Get latent encodings from the Trained Autoencoder

In [6]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        
        # batch_size : 16
        # Input size to the Autoencoder : [batch_size, 3, 32, 32]
        # Output size from the Autoencoder : [batch_size, 3, 32, 32]
        self.encoder = nn.Sequential(
            # 3 input channels(RGB), 12 output channels, 4x4 square convolution
            nn.Conv2d(in_channels = 3, out_channels = 12, kernel_size = 4, stride=2, padding=1), # Output size : [batch_size, 12, 16, 16]
            nn.ReLU(),
            nn.Conv2d(12, 24, 4, stride=2, padding=1), # Output size : [batch_size, 24, 8, 8]
            nn.ReLU(),
            nn.Conv2d(24, 48, 4, stride=2, padding=1), # Output size : [batch_size, 48, 4, 4],
            nn.ReLU(),
            
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(48, 24, 4, stride=2, padding=1), # Output size: [batch_size, 24, 8, 8]
            nn.ReLU(),  #TODO : try with LeakyReLU while decoding
            nn.ConvTranspose2d(24, 12, 4, stride=2, padding=1), # Output size: [batch_size, 12, 16, 16]
            nn.ReLU(),
            nn.ConvTranspose2d(12, 3, 4, stride=2, padding=1), # Output size: [batch_size, 3, 32, 32]
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [7]:
def create_model():
    autoencoder = Autoencoder()
    if torch.cuda.is_available():
        autoencoder = autoencoder.cuda()
        print("Model moved to GPU.")
    return autoencoder

In [8]:
# Create Autoencoder model
autoencoder = create_model()

print("Loading pretrained checkpoint...")
autoencoder.load_state_dict(torch.load("./weights/best_of_all_autoencoder.pkl"))

Model moved to GPU.
Loading pretrained checkpoint...


# Define and train the CNN classifier

In [9]:
class CNN_classifier(nn.Module):
    def __init__(self):
        super(CNN_classifier, self).__init__()
        nn.Conv2d(in_channels = 3, out_channels = 12, kernel_size = 4, stride=2, padding=1), # Output size : [batch_size, 12, 16, 16]

In [10]:
import torch.nn as nn
import torch.nn.functional as F


class CNN_classifier(nn.Module):
    def __init__(self):
#       Input size : [batch_size, 48, 4, 4]
        super(CNN_classifier, self).__init__()
        self.conv1 = nn.Conv2d(in_channels =48, out_channels = 24, kernel_size = 3, padding = 1) # Out size : [64, 24, 4, 4]
        self.pool = nn.MaxPool2d(2, 2) # Out size : [64, 24, 2, 2]
        self.fc1 = nn.Linear(96, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(-1, 24 * 2 * 2)
        x = self.fc1(x)
        return x


model = CNN_classifier()

In [11]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the classifier network

In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

cuda:0


CNN_classifier(
  (conv1): Conv2d(48, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=96, out_features=10, bias=True)
)

In [16]:
max_epoch = 10
running_loss = 0.0
for epoch in range(max_epoch):
    for i, data in enumerate(dataloders['train']):
        images, labels = data[0].to(device), data[1].to(device)
        images = Variable(images.cuda())

        autoencoder_encodings, _ = autoencoder(images)
#         print(encodings)

        # zero the parameter gradients
        optimizer.zero_grad()


        # forward + backward + optimize
        outputs = model(autoencoder_encodings)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')
print('Saving Model...')
if not os.path.exists('./weights'):
    os.mkdir('./weights')
torch.save(model.state_dict(), "./weights/cnn_classifier.pkl")

[1,   100] loss: 0.087
[1,   200] loss: 0.088
[1,   300] loss: 0.086
[1,   400] loss: 0.086
[1,   500] loss: 0.085
[1,   600] loss: 0.083
[2,   100] loss: 0.157
[2,   200] loss: 0.084
[2,   300] loss: 0.082
[2,   400] loss: 0.081
[2,   500] loss: 0.082
[2,   600] loss: 0.081
[3,   100] loss: 0.152
[3,   200] loss: 0.081
[3,   300] loss: 0.079
[3,   400] loss: 0.079
[3,   500] loss: 0.080
[3,   600] loss: 0.079
[4,   100] loss: 0.150
[4,   200] loss: 0.080
[4,   300] loss: 0.077
[4,   400] loss: 0.078
[4,   500] loss: 0.079
[4,   600] loss: 0.079
[5,   100] loss: 0.147
[5,   200] loss: 0.079
[5,   300] loss: 0.078
[5,   400] loss: 0.077
[5,   500] loss: 0.078
[5,   600] loss: 0.078
[6,   100] loss: 0.146
[6,   200] loss: 0.076
[6,   300] loss: 0.077
[6,   400] loss: 0.077
[6,   500] loss: 0.077
[6,   600] loss: 0.077
[7,   100] loss: 0.144
[7,   200] loss: 0.077
[7,   300] loss: 0.077
[7,   400] loss: 0.077
[7,   500] loss: 0.077
[7,   600] loss: 0.076
[8,   100] loss: 0.142
[8,   200] 

# Test the accuracy

In [17]:
correct = 0
total = 0
with torch.no_grad():
    for data in dataloders['val']:
        images, labels = data
        labels = labels.to(device)
        images = Variable(images.cuda())
        autoencoder_encodings, _ = autoencoder(images)
        outputs = model(autoencoder_encodings)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
#         print(predicted, labels)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 44 %


# Test the class wise accuracy


In [19]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [20]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in dataloders['val']:
        images, labels = data
        labels = labels.to(device)
        images = Variable(images.cuda())
        autoencoder_encodings, _ = autoencoder(images)
        outputs = model(autoencoder_encodings)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 61 %
Accuracy of   car : 65 %
Accuracy of  bird : 16 %
Accuracy of   cat : 27 %
Accuracy of  deer : 18 %
Accuracy of   dog : 27 %
Accuracy of  frog : 71 %
Accuracy of horse : 41 %
Accuracy of  ship : 60 %
Accuracy of truck : 42 %
